### Import necessary libraries, set options

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import patsy
import seaborn as sns
import statsmodels.api as sm
import warnings

from statsmodels.formula.api import glm

pd.set_option('display.max_columns', 125)
warnings.filterwarnings("ignore")

### Read in pickled datasets

In [ ]:
path_to_diss = os.path.join("path/to/dissertation/chapter")

path_to_data = os.path.join(path_to_diss, "path/to/data/processed-data")

##### Between-individual

In [ ]:
data_final = pd.read_pickle(path_to_data + "/pkl/data_final.pkl")
print(len(data_final))
data_final.head()

In [ ]:
data_final_coop = pd.read_pickle(path_to_data + "/pkl/data_final_coop.pkl")
print(len(data_final_coop))
data_final_coop.head()

In [ ]:
data_pid_version = pd.read_pickle(path_to_data + "/pkl/data_pid_version.pkl")
print(len(data_pid_version))
data_pid_version.head()

In [ ]:
data_pid = pd.read_pickle(path_to_data + "/pkl/data_pid.pkl")
print(len(data_pid))
data_pid.head()

##### Between-session

In [ ]:
session_data = pd.read_pickle(path_to_data + "/pkl/session_data.pkl")
print(len(session_data))
session_data.head()

In [ ]:
session_data_coop = pd.read_pickle(path_to_data + "/pkl/session_data_coop.pkl")
print(len(session_data_coop))
session_data_coop.head()

In [ ]:
session_data_pid_version = pd.read_pickle(path_to_data + "/pkl/session_data_pid_version.pkl")
print(len(session_data_pid_version))
session_data_pid_version.head()

##### Within-individual

In [ ]:
data_within = pd.read_pickle(path_to_data + "/pkl/data_within.pkl")
print(len(data_within))
data_within.head()

In [ ]:
data_within_coop = pd.read_pickle(path_to_data + "/pkl/data_within_coop.pkl")
print(len(data_within_coop))
data_within_coop.head()

In [ ]:
data_within_pid = pd.read_pickle(path_to_data + "/pkl/data_within_pid.pkl")
print(len(data_within_pid))
data_within_pid.head()

##### Within-session

In [ ]:
session_data_within = pd.read_pickle(path_to_data + "/pkl/session_data_within.pkl")
print(len(session_data_within))
session_data_within.head()

In [ ]:
session_data_within_coop = pd.read_pickle(path_to_data + "/pkl/session_data_within_coop.pkl")
print(len(session_data_within_coop))
session_data_within_coop.head()

In [ ]:
session_data_within_pid = pd.read_pickle(path_to_data + "/pkl/session_data_within_pid.pkl")
print(len(session_data_within_pid))
session_data_within_pid.head()

### Generate within-subjects interaction term

In [ ]:
data_within_pid['game1_int'] = data_within_pid['earned_v1'] * data_within_pid['equal_v1']
data_within_pid['game2_int'] = data_within_pid['earned_v2'] * data_within_pid['equal_v2']
data_within_pid['delta_int'] = data_within_pid['game2_int'] - data_within_pid['game1_int']

In [ ]:
data_within_coop['game1_int'] = data_within_coop['earned_v1'] * data_within_coop['equal_v1']
data_within_coop['game2_int'] = data_within_coop['earned_v2'] * data_within_coop['equal_v2']
data_within_coop['delta_int'] = data_within_coop['game2_int'] - data_within_coop['game1_int']

In [ ]:
session_data_within_pid['game1_int'] = session_data_within_pid['earned1'] * session_data_within_pid['equal1']
session_data_within_pid['game2_int'] = session_data_within_pid['earned2'] * session_data_within_pid['equal2']
session_data_within_pid['delta_int'] = session_data_within_pid['game2_int'] - session_data_within_pid['game1_int']

In [ ]:
session_data_within_coop['game1_int'] = session_data_within_coop['earned1'] * session_data_within_coop['equal1']
session_data_within_coop['game2_int'] = session_data_within_coop['earned2'] * session_data_within_coop['equal2']
session_data_within_coop['delta_int'] = session_data_within_coop['game2_int'] - session_data_within_coop['game1_int']

### Fairness ~ conditions

##### Overview

In [ ]:
print(np.mean(data_pid_version.f_score))
print()

print(data_pid_version.groupby('version')['f_score'].mean())
print()

print(data_pid_version.groupby('condition')['f_score'].mean())
print()

print(data_pid_version.groupby(['version', 'condition'])['f_score'].mean())

##### Figures

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))

for i in range(3):

    if i == 0:
        temp = data_pid_version[data_pid_version['version'] == 1].groupby('condition')['f_score'].mean().reset_index()
        ax[i].set_title("First game only")
    elif i == 1:
        temp = data_pid_version[data_pid_version['version'] == 2].groupby('condition')['f_score'].mean().reset_index()
        ax[i].set_title("Second game only")
    else:
        temp = data_pid_version.groupby('condition')['f_score'].mean().reset_index()
        ax[i].set_title("Pooled")

    groupedvalues = temp.sort_values(by = ['f_score']).reset_index(drop = True)
    #pal = sns.color_palette("Reds_d", len(groupedvalues))
    #rank = groupedvalues["f_score"].argsort().argsort()
    #sns.barplot(x='condition', y='f_score', data=groupedvalues, palette=np.array(pal[::-1])[rank], ax=ax[i])
    clrs = ['tomato', 'royalblue', 'limegreen', 'orange']
    sns.barplot(x = 'condition', y = 'f_score', data = groupedvalues, palette = clrs, ax = ax[i])
    for index, row in groupedvalues.iterrows():
        ax[i].text(row.name, row.f_score, round(row.f_score, 2), color = 'black', ha = "center")
    ax[i].set(xlabel = '', ylabel = 'Average fairness score')
    ax[i].set_xticklabels(['RU', 'EE', 'RE', 'EU'])
    ax[i].set(ylim=(0, 6))
    
fig.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA5.1.png"), 
            bbox_inches = 'tight', 
            pad_inches = 0.25)

##### Pairwise fairness and preference comparisons

In [ ]:
dic_more_fair = {
    'ee_vs_re': [],
    'ee_vs_eu': [],
    'ee_vs_ru': [],
    're_vs_eu': [],
    're_vs_ru': [],
    'eu_vs_ru': []
}

dic_prefer = {
    'ee_vs_re': [],
    'ee_vs_eu': [],
    'ee_vs_ru': [],
    're_vs_eu': [],
    're_vs_ru': [],
    'eu_vs_ru': []
}

In [ ]:
for index, row in data_pid.iterrows():
    if (row["earned1"] == 1 and row["equal1"] == 1 and row["earned2"] == 0 and row["equal2"] == 1) or \
       (row["earned1"] == 0 and row["equal1"] == 1 and row["earned2"] == 1 and row["equal2"] == 1):
            dic_more_fair['ee_vs_re'].append(row['more_fair'])
            dic_prefer['ee_vs_re'].append(row['prefer'])
    elif (row["earned1"] == 1 and row["equal1"] == 1 and row["earned2"] == 1 and row["equal2"] == 0) or \
       (row["earned1"] == 1 and row["equal1"] == 0 and row["earned2"] == 1 and row["equal2"] == 1):
            dic_more_fair['ee_vs_eu'].append(row['more_fair'])
            dic_prefer['ee_vs_eu'].append(row['prefer'])
    elif (row["earned1"] == 1 and row["equal1"] == 1 and row["earned2"] == 0 and row["equal2"] == 0) or \
       (row["earned1"] == 0 and row["equal1"] == 0 and row["earned2"] == 1 and row["equal2"] == 1):
            dic_more_fair['ee_vs_ru'].append(row['more_fair'])
            dic_prefer['ee_vs_ru'].append(row['prefer'])  
    elif (row["earned1"] == 0 and row["equal1"] == 1 and row["earned2"] == 1 and row["equal2"] == 0) or \
       (row["earned1"] == 1 and row["equal1"] == 0 and row["earned2"] == 0 and row["equal2"] == 1):
            dic_more_fair['re_vs_eu'].append(row['more_fair'])
            dic_prefer['re_vs_eu'].append(row['prefer'])
    elif (row["earned1"] == 0 and row["equal1"] == 1 and row["earned2"] == 0 and row["equal2"] == 0) or \
       (row["earned1"] == 0 and row["equal1"] == 0 and row["earned2"] == 0 and row["equal2"] == 1):
            dic_more_fair['re_vs_ru'].append(row['more_fair'])
            dic_prefer['re_vs_ru'].append(row['prefer'])       
    elif (row["earned1"] == 1 and row["equal1"] == 0 and row["earned2"] == 0 and row["equal2"] == 0) or \
       (row["earned1"] == 0 and row["equal1"] == 0 and row["earned2"] == 1 and row["equal2"] == 0):
            dic_more_fair['eu_vs_ru'].append(row['more_fair'])
            dic_prefer['eu_vs_ru'].append(row['prefer'])

In [ ]:
for key in dic_more_fair.keys():
    temp_dict = {}
    for condition in np.unique(dic_more_fair[key]):
        if condition != '':
            temp_dict[condition] = [dic_more_fair[key].count(condition)]
    dic_more_fair[key] = temp_dict
    tot = 0
    for key2 in dic_more_fair[key].keys():
        tot += dic_more_fair[key][key2][0]
    for key2 in dic_more_fair[key].keys():
        raw_count = dic_more_fair[key][key2][0]
        perc = round(raw_count / tot * 100, 2)
        dic_more_fair[key][key2].append(perc)
    
dic_more_fair

In [ ]:
for key in dic_prefer.keys():
    temp_dict = {}
    for condition in np.unique(dic_prefer[key]):
        if condition != '':
            temp_dict[condition] = [dic_prefer[key].count(condition)]
    dic_prefer[key] = temp_dict
    tot = 0
    for key2 in dic_prefer[key].keys():
        tot += dic_prefer[key][key2][0]
    for key2 in dic_prefer[key].keys():
        raw_count = dic_prefer[key][key2][0]
        perc = round(raw_count / tot * 100, 2)
        dic_prefer[key][key2].append(perc)
    
dic_prefer

##### Between-individual

In [ ]:
# Between first only

y, X = patsy.dmatrices(
    'f_score ~ earned * equal', 
    data_pid_version[data_pid_version['version'] == 1],
    return_type = 'dataframe'
)

session_c = data_pid_version[(data_pid_version['version'] == 1)                           &
                             (data_pid_version['version'] == data_pid_version['version']) &
                             (data_pid_version['f_score'] == data_pid_version['f_score']) &
                             (data_pid_version['earned']  == data_pid_version['earned'])  &
                             (data_pid_version['equal']   == data_pid_version['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
# Between second only

y, X = patsy.dmatrices(
    'f_score ~ earned * equal', 
    data_pid_version[data_pid_version['version'] == 2],
    return_type = 'dataframe'
)

session_c = data_pid_version[(data_pid_version['version'] == 2)                           &
                             (data_pid_version['version'] == data_pid_version['version']) &
                             (data_pid_version['f_score'] == data_pid_version['f_score']) &
                             (data_pid_version['earned']  == data_pid_version['earned'])  &
                             (data_pid_version['equal']   == data_pid_version['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
# Between pooled

y, X = patsy.dmatrices(
    'f_score ~ earned * equal + C(version)', 
    data_pid_version,
    return_type = 'dataframe'
)

session_c = data_pid_version[(data_pid_version['version'] == data_pid_version['version']) &
                             (data_pid_version['f_score'] == data_pid_version['f_score']) &
                             (data_pid_version['earned']  == data_pid_version['earned'])  &
                             (data_pid_version['equal']   == data_pid_version['equal'])]['session_no']

pid_c     = data_pid_version[(data_pid_version['version'] == data_pid_version['version']) &
                             (data_pid_version['f_score'] == data_pid_version['f_score']) &
                             (data_pid_version['earned']  == data_pid_version['earned'])  &
                             (data_pid_version['equal']   == data_pid_version['equal'])]['pid']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
# Between version interaction

y, X = patsy.dmatrices(
    'f_score ~ earned * equal * C(version)', 
    data_pid_version,
    return_type = 'dataframe'
)

session_c = data_pid_version[(data_pid_version['version'] == data_pid_version['version']) &
                             (data_pid_version['f_score'] == data_pid_version['f_score']) &
                             (data_pid_version['earned']  == data_pid_version['earned'])  &
                             (data_pid_version['equal']   == data_pid_version['equal'])]['session_no']

pid_c     = data_pid_version[(data_pid_version['version'] == data_pid_version['version']) &
                             (data_pid_version['f_score'] == data_pid_version['f_score']) &
                             (data_pid_version['earned']  == data_pid_version['earned'])  &
                             (data_pid_version['equal']   == data_pid_version['equal'])]['pid']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
# Between first only + change_in_score

y, X = patsy.dmatrices(
    'f_score ~ earned * equal + change_in_score', 
    data_pid_version[data_pid_version['version'] == 1],
    return_type = 'dataframe'
)

session_c = data_pid_version[(data_pid_version['version'] == 1)                                           &
                             (data_pid_version['change_in_score'] == data_pid_version['change_in_score']) &
                             (data_pid_version['version']         == data_pid_version['version'])         &
                             (data_pid_version['f_score']         == data_pid_version['f_score'])         &
                             (data_pid_version['earned']          == data_pid_version['earned'])          &
                             (data_pid_version['equal']           == data_pid_version['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
# Between second only + change_in_score

y, X = patsy.dmatrices(
    'f_score ~ earned * equal + change_in_score', 
    data_pid_version[data_pid_version['version'] == 2],
    return_type = 'dataframe'
)

session_c = data_pid_version[(data_pid_version['version'] == 2)                                           &
                             (data_pid_version['change_in_score'] == data_pid_version['change_in_score']) &
                             (data_pid_version['version']         == data_pid_version['version'])         &
                             (data_pid_version['f_score']         == data_pid_version['f_score'])         &
                             (data_pid_version['earned']          == data_pid_version['earned'])          &
                             (data_pid_version['equal']           == data_pid_version['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
# Between pooled + change_in_score

y, X = patsy.dmatrices(
    'f_score ~ earned * equal + C(version) + change_in_score', 
    data_pid_version,
    return_type = 'dataframe'
)

session_c = data_pid_version[(data_pid_version['change_in_score'] == data_pid_version['change_in_score']) &
                             (data_pid_version['version']         == data_pid_version['version'])         &
                             (data_pid_version['f_score']         == data_pid_version['f_score'])         &
                             (data_pid_version['earned']          == data_pid_version['earned'])          &
                             (data_pid_version['equal']           == data_pid_version['equal'])]['session_no']

pid_c     = data_pid_version[(data_pid_version['change_in_score'] == data_pid_version['change_in_score']) &
                             (data_pid_version['version']         == data_pid_version['version'])         &
                             (data_pid_version['f_score']         == data_pid_version['f_score'])         &
                             (data_pid_version['earned']          == data_pid_version['earned'])          &
                             (data_pid_version['equal']           == data_pid_version['equal'])]['pid']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
# Between version interaction + change_in_score

y, X = patsy.dmatrices(
    'f_score ~ earned * equal * C(version) + change_in_score', 
    data_pid_version,
    return_type = 'dataframe'
)

session_c = data_pid_version[(data_pid_version['change_in_score'] == data_pid_version['change_in_score']) &
                             (data_pid_version['version']         == data_pid_version['version'])         &
                             (data_pid_version['f_score']         == data_pid_version['f_score'])         &
                             (data_pid_version['earned']          == data_pid_version['earned'])          &
                             (data_pid_version['equal']           == data_pid_version['equal'])]['session_no']

pid_c     = data_pid_version[(data_pid_version['change_in_score'] == data_pid_version['change_in_score']) &
                             (data_pid_version['version']         == data_pid_version['version'])         &
                             (data_pid_version['f_score']         == data_pid_version['f_score'])         &
                             (data_pid_version['earned']          == data_pid_version['earned'])          &
                             (data_pid_version['equal']           == data_pid_version['equal'])]['pid']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Between-session

In [ ]:
y, X = patsy.dmatrices(
    'f_score ~ earned * equal', 
    session_data_pid_version[session_data_pid_version['version'] == 1],
    return_type = 'dataframe'
)

ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
y, X = patsy.dmatrices(
    'f_score ~ earned * equal', 
    session_data_pid_version[session_data_pid_version['version'] == 2],
    return_type = 'dataframe'
)

ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
y, X = patsy.dmatrices(
    'f_score ~ earned * equal + C(version)', 
    session_data_pid_version,
    return_type = 'dataframe'
)

session_c = session_data_pid_version[
    (session_data_pid_version['version'] == session_data_pid_version['version']) &
    (session_data_pid_version['f_score'] == session_data_pid_version['f_score']) &
    (session_data_pid_version['earned']  == session_data_pid_version['earned'])  &
    (session_data_pid_version['equal']   == session_data_pid_version['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'f_score ~ earned * equal * C(version)', 
    session_data_pid_version,
    return_type = 'dataframe'
)

session_c = session_data_pid_version[
    (session_data_pid_version['version'] == session_data_pid_version['version']) &
    (session_data_pid_version['f_score'] == session_data_pid_version['f_score']) &
    (session_data_pid_version['earned']  == session_data_pid_version['earned'])  &
    (session_data_pid_version['equal']   == session_data_pid_version['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Within-individual

In [ ]:
# y, X = patsy.dmatrices(
#     'delta_f_score ~ delta_earned + delta_equal + delta_int', 
#     # 'delta_f_score ~ delta_earned * delta_equal'
#     data_within_pid,
#     return_type = 'dataframe'
# )

# session_c = data_within_pid[(data_within_pid['delta_f_score'] == data_within_pid['delta_f_score']) &
#                             (data_within_pid['delta_earned']  == data_within_pid['delta_earned'])  &
#                             (data_within_pid['delta_equal']   == data_within_pid['delta_equal'])]['session_no']

# ols = sm.OLS(y, X)
# ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
temp = data_within_pid.copy()

temp['delta_earned'] = temp['delta_earned'].apply(lambda row: 2 if row == -1 else row)
temp['delta_equal']  = temp['delta_equal'].apply(lambda row: 2 if row == -1 else row)

y, X = patsy.dmatrices(
    'delta_f_score ~ C(delta_earned) * C(delta_equal)',
    temp,
    return_type = 'dataframe'
)

session_c = temp[(temp['delta_f_score'] == temp['delta_f_score']) &
                 (temp['delta_earned']  == temp['delta_earned'])  &
                 (temp['delta_equal']   == temp['delta_equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'delta_f_score ~ delta_earned + delta_equal + delta_int + delta_change_in_score',
    # 'delta_f_score ~ delta_earned * delta_equal + delta_change_in_score'
    data_within_pid,
    return_type = 'dataframe'
)

session_c = data_within_pid[
    (data_within_pid['delta_change_in_score'] == data_within_pid['delta_change_in_score']) &
    (data_within_pid['delta_f_score']         == data_within_pid['delta_f_score'])         &
    (data_within_pid['delta_earned']          == data_within_pid['delta_earned'])          &
    (data_within_pid['delta_equal']           == data_within_pid['delta_equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Within-session

In [ ]:
y, X = patsy.dmatrices(
    'delta_f_score ~ delta_earned + delta_equal + delta_int',
    # 'delta_f_score ~ delta_earned * delta_equal'
    session_data_within_pid,
    return_type = 'dataframe'
)

ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
temp = session_data_within_pid.copy()

temp['delta_earned'] = temp['delta_earned'].apply(lambda row: 2 if row == -1 else row)
temp['delta_equal']  = temp['delta_equal'].apply(lambda row: 2 if row == -1 else row)

y, X = patsy.dmatrices(
    'delta_f_score ~ C(delta_earned) * C(delta_equal)',
    temp,
    return_type = 'dataframe'
)

ols = sm.OLS(y, X)
ols.fit().summary()

### Cooperation ~ conditions

##### Overview

In [ ]:
print(np.mean(data_final_coop.coopChoice))
print()

print(data_final_coop.groupby('version')['coopChoice'].mean())
print()

print(data_final_coop.groupby('round')['coopChoice'].mean())
print()

In [ ]:
print(data_final_coop.groupby('condition')['coopChoice'].mean())
print()

print(data_final_coop.groupby(['version', 'condition'])['coopChoice'].mean())

##### Figures

In [ ]:
data_final_coop.groupby(['round', 'version']).mean()['coopChoice'].unstack().plot(style = ['-', '--'],
                                                                                  color = 'black')
plt.xlabel('Round')
plt.ylabel('Average Cooperation')
plt.title('Average Cooperation by Round')
plt.legend(['First game', 'Second game'])
plt.xticks(np.arange(1, 11, step=1))

plt.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA5.2.png"))

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
clrs = ['royalblue', 'orange', 'limegreen', 'tomato']

for i in range(3):
    if i == 0:
        temp = data_final_coop[
            data_final_coop['version'] == 1].groupby(['round',
                                                      'condition']).mean()['coopChoice'].unstack().plot(color = clrs, 
                                                                                                        ax = ax[i])
        ax[i].set_title('First game only')
        ax[i].set_xticks(np.arange(1, 10, step = 1))
    elif i == 1:
        temp = data_final_coop[
            data_final_coop['version'] == 2].groupby(['round',
                                                      'condition']).mean()['coopChoice'].unstack().plot(color = clrs,
                                                                                                        ax = ax[i])
        ax[i].set_title('Second game only')
        ax[i].set_xticks(np.arange(1, 11, step = 1))
    else:
        temp = data_final_coop[
            data_final_coop['round'] != 10].groupby(['round',
                                                     'condition']).mean()['coopChoice'].unstack().plot(color = clrs,
                                                                                                       ax = ax[i])
        ax[i].set_title('Pooled')
        ax[i].set_xticks(np.arange(1, 10, step = 1))
    ax[i].set_xlabel('Round')
    ax[i].set_ylabel('Average Cooperation')
    ax[i].set_yticks(np.arange(0.35, 0.85, step = 0.05))
    ax[i].legend(['EE', 'EU', 'RE', 'RU'])
    
fig.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA5.3.png"), 
            bbox_inches = 'tight', 
            pad_inches = 0.25)

In [ ]:
data = {'Experimental condition': ['RU to RU', 'RU to EE', 'RU to RE', 'RU to EU', 
                                   'EE to RU', 'EE to EE', 'EE to RE', 'EE to EU', 
                                   'RE to RU', 'RE to EE', 'RE to RE', 'RE to EU', 
                                   'EU to RU', 'EU to EE', 'EU to RE', 'EU to EU'],
        'Change in fairness score': [ 0.4393,  0.2451,  0.4959,  1.5124, 
                                     -0.1759,  0.0374,  0.2162,  1.4580, 
                                     -0.3945, -0.4087,  0.5403,  0.7417, 
                                     -1.2234, -1.0642, -1.0096, -0.2833],
        'Change in cooperation': [ 0.0332, -0.0404, -0.0230, -0.0192, 
                                   0.0510, -0.0232, -0.0014, -0.0266, 
                                   0.0103, -0.0888,  0.0175, -0.0174, 
                                   0.0016,  0.0674,  0.0588,  0.0002]}

df = pd.DataFrame(data)

df.columns = ['g', 'x', 'y']

x = list(df['x'])
y = list(df['y'])

p1 = np.polyfit(x, y, 1)

plt.rcParams["figure.figsize"] = [10, 8]

plt.xlim(-1.7, 1.7)
plt.ylim(-0.095, 0.095)

plt.xlabel('Change in perceived fairness score')
plt.ylabel('Change in probability of cooperating')

plt.scatter(x, y)
xlims = plt.xlim()
x.insert(0, xlims[0])
y.insert(0, np.polyval(p1, xlims[0]))
x.append(xlims[1])
y.append(np.polyval(p1, xlims[1]))
plt.plot(x, np.polyval(p1,x), 'r-', linewidth = 1.5)
plt.xlim(xlims)

for line in range(0, df.shape[0]):
     plt.text(df.x[line]-0.1, 
              df.y[line]+0.0025, 
              df.g[line], 
              horizontalalignment='left', 
              size='small', 
              color='black')

plt.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA4.1.png"))        

plt.show()

##### Cooperation history

In [ ]:
def get_coop_history(row):
    return list(row)[1:]

In [ ]:
df_wide_v1 = data_final_coop[data_final_coop['version'] == 1][
    ['pid', 'round', 'coopChoice']
].pivot(index = 'pid', columns = 'round', values = 'coopChoice').reset_index().rename_axis(None, axis = 1)

df_wide_v1.head()

In [ ]:
df_wide_v2 = data_final_coop[data_final_coop['version'] == 2][
    ['pid', 'round', 'coopChoice']
].pivot(index = 'pid', columns = 'round', values = 'coopChoice').reset_index().rename_axis(None, axis = 1)

df_wide_v2.head()

In [ ]:
df_wide_v1['coop_history'] = df_wide_v1.apply(get_coop_history, axis = 1)
df_wide_v1.head()

In [ ]:
df_wide_v2['coop_history'] = df_wide_v2.apply(get_coop_history, axis = 1)
df_wide_v2.head()

In [ ]:
d1 = {}
for ch in df_wide_v1['coop_history']:
    if str(ch) in d1.keys():
        d1[str(ch)] += 1
    else:
        d1[str(ch)] = 1
d1

In [ ]:
d2 = {}
for ch in df_wide_v2['coop_history']:
    if str(ch) in d2.keys():
        d2[str(ch)] += 1
    else:
        d2[str(ch)] = 1
d2

In [ ]:
ni1 = 0
o1  = 0

for ch in df_wide_v1['coop_history']:
    if sorted(ch, reverse = True) == ch:
        ni1 += 1
    else:
        o1 += 1
        
ni1/(ni1+o1)

In [ ]:
ni2 = 0
o2  = 0

for ch in df_wide_v2['coop_history']:
    if sorted(ch, reverse = True) == ch:
        ni2 += 1
    else:
        o2 += 1
        
ni2/(ni2+o2)

##### Trust as a predictor of cooperation in the second game

In [ ]:
temp = data_final_coop.groupby(['session_no', 
                                'version']).mean().reset_index()[['session_no', 
                                                                  'version',
                                                                  'coopChoice', 
                                                                  'trust_score']]

temp1 = temp[temp['version'] == 1]
temp2 = temp[temp['version'] == 2]

temp = temp1.merge(right = temp2, how = "inner", on = ["session_no"], suffixes = ("_v1", "_v2"))
temp.head()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice_v2 ~ trust_score_v1 + coopChoice_v1', 
    temp,
    return_type = 'dataframe'
)

ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
y, X = patsy.dmatrices(
    'trust_score_v1 ~ coopChoice_v1', 
    temp,
    return_type = 'dataframe'
)

ols = sm.OLS(y, X)
ols.fit().summary()

In [ ]:
np.corrcoef(temp['trust_score_v1'], temp['coopChoice_v1'])

##### Average number of neighbors

In [ ]:
def count_num_neighbors(row):
    return len(row['neighborsList'])

In [ ]:
np.mean(data_final_coop.apply(count_num_neighbors, axis = 1))

##### Average game scores

In [ ]:
print(np.mean(data_pid['score1']))
print(np.mean(data_pid['score2']))
print(np.mean(data_pid['score3']))

##### Why people cooperate

In [ ]:
dic_why = {
    "altruism": 0,
    "encourage": 0,
    "choseA": 0,
    "choseB": 0,
    "equal": 0,
    "more": 0,
    "less": 0,
    "fair": 0,
    "notfair": 0,
    "other": 0,
    "other_text": []
}

In [ ]:
for index, row in data_pid.iterrows():
    if row["why_coop_list"] == row["why_coop_list"]:
        if "other" in list(row["why_coop_list"]):
            dic_why["other_text"].append(row["why_coop_other"])
        for e in row["why_coop_list"]:
            dic_why[e] += 1

In [ ]:
dict((key,value) for key, value in dic_why.items() if key != "other_text")

In [ ]:
print(len(dic_why["other_text"]) == dic_why["other"])

dic_why["other_text"]

##### Between-individual (using Logit here but same conclusions with OLS)

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(round)', 
    data_final_coop[data_final_coop['version'] == 1],
    return_type = 'dataframe'
)

session_c = data_final_coop[(data_final_coop['version']    == 1)                             &
                            (data_final_coop['version']    == data_final_coop['version'])    &
                            (data_final_coop['round']      == data_final_coop['round'])      &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['earned']     == data_final_coop['earned'])     &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['session_no']

pid_c     = data_final_coop[(data_final_coop['version']    == 1)                             &
                            (data_final_coop['version']    == data_final_coop['version'])    &
                            (data_final_coop['round']      == data_final_coop['round'])      &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['earned']     == data_final_coop['earned'])     &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['pid']

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(round)', 
    data_final_coop[data_final_coop['version'] == 2],
    return_type = 'dataframe'
)

session_c = data_final_coop[(data_final_coop['version']    == 2)                             &
                            (data_final_coop['version']    == data_final_coop['version'])    &
                            (data_final_coop['round']      == data_final_coop['round'])      &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['earned']     == data_final_coop['earned'])     &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['session_no']

pid_c     = data_final_coop[(data_final_coop['version']    == 2)                             &
                            (data_final_coop['version']    == data_final_coop['version'])    &
                            (data_final_coop['round']      == data_final_coop['round'])      &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['earned']     == data_final_coop['earned'])     &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['pid']

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(version) + C(round)', 
    data_final_coop[data_final_coop['round'] != 10],
    return_type = 'dataframe'
)

session_c = data_final_coop[(data_final_coop['round']      != 10)                            &
                            (data_final_coop['version']    == data_final_coop['version'])    &
                            (data_final_coop['round']      == data_final_coop['round'])      &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['earned']     == data_final_coop['earned'])     &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['session_no']

pid_c     = data_final_coop[(data_final_coop['round']      != 10)                            &
                            (data_final_coop['version']    == data_final_coop['version'])    &
                            (data_final_coop['round']      == data_final_coop['round'])      &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['earned']     == data_final_coop['earned'])     &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['pid']

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal * C(version) + C(round)', 
    data_final_coop[data_final_coop['round'] != 10],
    return_type = 'dataframe'
)

session_c = data_final_coop[(data_final_coop['round']      != 10)                            &
                            (data_final_coop['version']    == data_final_coop['version'])    &
                            (data_final_coop['round']      == data_final_coop['round'])      &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['earned']     == data_final_coop['earned'])     &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['session_no']

pid_c     = data_final_coop[(data_final_coop['round']      != 10)                            &
                            (data_final_coop['version']    == data_final_coop['version'])    &
                            (data_final_coop['round']      == data_final_coop['round'])      &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['earned']     == data_final_coop['earned'])     &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['pid']

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(round) + scoreBeforeCoop', 
    data_final_coop[data_final_coop['version'] == 1],
    return_type = 'dataframe'
)

session_c = data_final_coop[(data_final_coop['version']    == 1)                                  &
                            (data_final_coop['version']    == data_final_coop['version'])         &
                            (data_final_coop['round']      == data_final_coop['round'])           &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice'])      &
                            (data_final_coop['earned']     == data_final_coop['earned'])          &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['session_no']

pid_c     = data_final_coop[(data_final_coop['version']    == 1)                                  &
                            (data_final_coop['version']    == data_final_coop['version'])         &
                            (data_final_coop['round']      == data_final_coop['round'])           &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice'])      &
                            (data_final_coop['earned']     == data_final_coop['earned'])          &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['pid']

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(round) + scoreBeforeCoop', 
    data_final_coop[data_final_coop['version'] == 2],
    return_type = 'dataframe'
)

session_c = data_final_coop[(data_final_coop['version']    == 2)                                  &
                            (data_final_coop['version']    == data_final_coop['version'])         &
                            (data_final_coop['round']      == data_final_coop['round'])           &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice'])      &
                            (data_final_coop['earned']     == data_final_coop['earned'])          &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['session_no']

pid_c     = data_final_coop[(data_final_coop['version']    == 2)                                  &
                            (data_final_coop['version']    == data_final_coop['version'])         &
                            (data_final_coop['round']      == data_final_coop['round'])           &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice'])      &
                            (data_final_coop['earned']     == data_final_coop['earned'])          &
                            (data_final_coop['equal']      == data_final_coop['equal'])]['pid']

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(version) + C(round) + scoreBeforeCoop', 
    data_final_coop,
    return_type = 'dataframe'
)

session_c = data_final_coop[(data_final_coop['scoreBeforeCoop'] == data_final_coop['scoreBeforeCoop']) &
                            (data_final_coop['version']         == data_final_coop['version'])         &
                            (data_final_coop['round']           == data_final_coop['round'])           &
                            (data_final_coop['coopChoice']      == data_final_coop['coopChoice'])      &
                            (data_final_coop['earned']          == data_final_coop['earned'])          &
                            (data_final_coop['equal']           == data_final_coop['equal'])]['session_no']

pid_c     = data_final_coop[(data_final_coop['scoreBeforeCoop'] == data_final_coop['scoreBeforeCoop']) &
                            (data_final_coop['version']         == data_final_coop['version'])         &
                            (data_final_coop['round']           == data_final_coop['round'])           &
                            (data_final_coop['coopChoice']      == data_final_coop['coopChoice'])      &
                            (data_final_coop['earned']          == data_final_coop['earned'])          &
                            (data_final_coop['equal']           == data_final_coop['equal'])]['pid']

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal * C(version) + C(round) + scoreBeforeCoop', 
    data_final_coop,
    return_type = 'dataframe'
)

session_c = data_final_coop[(data_final_coop['scoreBeforeCoop'] == data_final_coop['scoreBeforeCoop']) &
                            (data_final_coop['version']         == data_final_coop['version'])         &
                            (data_final_coop['round']           == data_final_coop['round'])           &
                            (data_final_coop['coopChoice']      == data_final_coop['coopChoice'])      &
                            (data_final_coop['earned']          == data_final_coop['earned'])          &
                            (data_final_coop['equal']           == data_final_coop['equal'])]['session_no']

pid_c     = data_final_coop[(data_final_coop['scoreBeforeCoop'] == data_final_coop['scoreBeforeCoop']) &
                            (data_final_coop['version']         == data_final_coop['version'])         &
                            (data_final_coop['round']           == data_final_coop['round'])           &
                            (data_final_coop['coopChoice']      == data_final_coop['coopChoice'])      &
                            (data_final_coop['earned']          == data_final_coop['earned'])          &
                            (data_final_coop['equal']           == data_final_coop['equal'])]['pid']

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Between-session

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(round)', 
    session_data_coop[session_data_coop['version'] == 1],
    return_type = 'dataframe'
)

session_c = session_data_coop[(session_data_coop['version']    == 1)                               &
                              (session_data_coop['version']    == session_data_coop['version'])    &
                              (session_data_coop['round']      == session_data_coop['round'])      &
                              (session_data_coop['coopChoice'] == session_data_coop['coopChoice']) &
                              (session_data_coop['earned']     == session_data_coop['earned'])     &
                              (session_data_coop['equal']      == session_data_coop['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(version) + C(round)', 
    session_data_coop[session_data_coop['version'] == 2],
    return_type = 'dataframe'
)

session_c = session_data_coop[(session_data_coop['version']    == 2)                               &
                              (session_data_coop['version']    == session_data_coop['version'])    &
                              (session_data_coop['round']      == session_data_coop['round'])      &
                              (session_data_coop['coopChoice'] == session_data_coop['coopChoice']) &
                              (session_data_coop['earned']     == session_data_coop['earned'])     &
                              (session_data_coop['equal']      == session_data_coop['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal + C(version) + C(round)', 
    session_data_coop[session_data_coop['round'] != 10],
    return_type = 'dataframe'
)

session_c = session_data_coop[(session_data_coop['round']      != 10)                              &
                              (session_data_coop['version']    == session_data_coop['version'])    &
                              (session_data_coop['round']      == session_data_coop['round'])      &
                              (session_data_coop['coopChoice'] == session_data_coop['coopChoice']) &
                              (session_data_coop['earned']     == session_data_coop['earned'])     &
                              (session_data_coop['equal']      == session_data_coop['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ earned * equal * C(version) + C(round)', 
    session_data_coop[session_data_coop['round'] != 10],
    return_type = 'dataframe'
)

session_c = session_data_coop[(session_data_coop['round']      != 10)                              &
                              (session_data_coop['version']    == session_data_coop['version'])    &
                              (session_data_coop['round']      == session_data_coop['round'])      &
                              (session_data_coop['coopChoice'] == session_data_coop['coopChoice']) &
                              (session_data_coop['earned']     == session_data_coop['earned'])     &
                              (session_data_coop['equal']      == session_data_coop['equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Within-individual

In [ ]:
# y, X = patsy.dmatrices(
#     'delta_coopChoice ~ delta_earned + delta_equal + delta_int + C(round)', 
#     # 'delta_coopChoice ~ delta_earned * delta_equal + C(round)'
#     data_within_coop,
#     return_type = 'dataframe'
# )

# session_c = data_within_coop[(data_within_coop['round']            == data_within_coop['round'])            &
#                              (data_within_coop['delta_coopChoice'] == data_within_coop['delta_coopChoice']) &
#                              (data_within_coop['delta_earned']     == data_within_coop['delta_earned'])     &
#                              (data_within_coop['delta_equal']      == data_within_coop['delta_equal'])]['session_no']

# pid_c     = data_within_coop[(data_within_coop['round']            == data_within_coop['round'])            &
#                              (data_within_coop['delta_coopChoice'] == data_within_coop['delta_coopChoice']) &
#                              (data_within_coop['delta_earned']     == data_within_coop['delta_earned'])     &
#                              (data_within_coop['delta_equal']      == data_within_coop['delta_equal'])]['pid']

# ols = sm.OLS(y, X)
# ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
temp = data_within_coop.copy()

temp['delta_earned'] = temp['delta_earned'].apply(lambda row: 2 if row == -1 else row)
temp['delta_equal']  = temp['delta_equal'].apply(lambda row: 2 if row == -1 else row)

y, X = patsy.dmatrices(
    'delta_coopChoice ~ C(delta_earned) * C(delta_equal) + C(round)', 
    temp,
    return_type = 'dataframe'
)

session_c = temp[(temp['round']            == temp['round'])            &
                 (temp['delta_coopChoice'] == temp['delta_coopChoice']) &
                 (temp['delta_earned']     == temp['delta_earned'])     &
                 (temp['delta_equal']      == temp['delta_equal'])]['session_no']

pid_c     = temp[(temp['round']            == temp['round'])            &
                 (temp['delta_coopChoice'] == temp['delta_coopChoice']) &
                 (temp['delta_earned']     == temp['delta_earned'])     &
                 (temp['delta_equal']      == temp['delta_equal'])]['pid']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'delta_coopChoice ~ delta_earned + delta_equal + delta_int + delta_scoreBeforeCoop + num_other + C(round)', 
    # 'delta_coopChoice ~ delta_earned * delta_equal + delta_scoreBeforeCoop + C(round)'
    data_within_coop,
    return_type = 'dataframe'
)

session_c = data_within_coop[
    (data_within_coop['delta_scoreBeforeCoop'] == data_within_coop['delta_scoreBeforeCoop']) &
    (data_within_coop['round']                 == data_within_coop['round'])                 &
    (data_within_coop['delta_coopChoice']      == data_within_coop['delta_coopChoice'])      &
    (data_within_coop['delta_earned']          == data_within_coop['delta_earned'])          &
    (data_within_coop['delta_equal']           == data_within_coop['delta_equal'])           &
    (data_within_coop['num_other']             == data_within_coop['num_other'])]['session_no']

pid_c = data_within_coop[
    (data_within_coop['delta_scoreBeforeCoop'] == data_within_coop['delta_scoreBeforeCoop']) &
    (data_within_coop['round']                 == data_within_coop['round'])                 &
    (data_within_coop['delta_coopChoice']      == data_within_coop['delta_coopChoice'])      &
    (data_within_coop['delta_earned']          == data_within_coop['delta_earned'])          &
    (data_within_coop['delta_equal']           == data_within_coop['delta_equal'])           &
    (data_within_coop['num_other']             == data_within_coop['num_other'])]['pid']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Within-session

In [ ]:
y, X = patsy.dmatrices(
    'delta_coopChoice ~ delta_earned + delta_equal + delta_int + C(round)',
    # 'delta_coopChoice ~ delta_earned * delta_equal + C(round)'
    session_data_within_coop,
    return_type = 'dataframe'
)

session_c = session_data_within_coop[
    (session_data_within_coop['round']            == session_data_within_coop['round'])            &
    (session_data_within_coop['delta_coopChoice'] == session_data_within_coop['delta_coopChoice']) &
    (session_data_within_coop['delta_earned']     == session_data_within_coop['delta_earned'])     &
    (session_data_within_coop['delta_equal']      == session_data_within_coop['delta_equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
temp = session_data_within_coop.copy()

temp['delta_earned'] = temp['delta_earned'].apply(lambda row: 2 if row == -1 else row)
temp['delta_equal']  = temp['delta_equal'].apply(lambda row: 2 if row == -1 else row)

y, X = patsy.dmatrices(
    'delta_coopChoice ~ C(delta_earned) * C(delta_equal)',
    temp,
    return_type = 'dataframe'
)

session_c = temp[(temp['round']            == temp['round'])            &
                 (temp['delta_coopChoice'] == temp['delta_coopChoice']) &
                 (temp['delta_earned']     == temp['delta_earned'])     &
                 (temp['delta_equal']      == temp['delta_equal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()